<a href="https://colab.research.google.com/github/Stitaprajna/Practise-codes/blob/main/Stock%20Data%20Grabber%20Daily.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [69]:
!pip install yfinance
!pip install pandas_ta
import yfinance as yf
import pandas_ta as ta
import pandas as pd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [70]:
def squeeze_momentum(df):
        # parameter setup (default values in the original indicator)
        length = 20
        mult = 2
        length_KC = 20
        mult_KC = 1.5

        # calculate Bollinger Bands
        # moving average
        m_avg = df['close'].rolling(window=length).mean()
        # standard deviation
        m_std = df['close'].rolling(window=length).std(ddof=0)
        # upper Bollinger Bands
        df['upper_BB'] = m_avg + mult * m_std
        # lower Bollinger Bands
        df['lower_BB'] = m_avg - mult * m_std

        # calculate Keltner Channel
        # first we need to calculate True Range
        df['tr0'] = abs(df["high"] - df["low"])
        df['tr1'] = abs(df["high"] - df["close"].shift())
        df['tr2'] = abs(df["low"] - df["close"].shift())
        df['tr'] = df[['tr0', 'tr1', 'tr2']].max(axis=1)
        # moving average of the TR
        range_ma = df['tr'].rolling(window=length_KC).mean()
        # upper Keltner Channel
        df['upper_KC'] = m_avg + range_ma * mult_KC
        # lower Keltner Channel
        df['lower_KC'] = m_avg - range_ma * mult_KC

        # calculate bar value
        highest = df['high'].rolling(window = length_KC).max()
        lowest = df['low'].rolling(window = length_KC).min()
        m1 = (highest + lowest)/2
        df['value'] = (df['close'] - (m1 + m_avg)/2)
        fit_y = np.array(range(0,length_KC))
        df['value'] = df['value'].rolling(window = length_KC).apply(lambda x:
                            np.polyfit(fit_y, x, 1)[0] * (length_KC-1) +
                            np.polyfit(fit_y, x, 1)[1], raw=True)
        # df['value'] = df['value'] * 100 / df['value'].max()
        # check for 'squeeze'
        df['squeeze_on'] = (df['lower_BB'] > df['lower_KC']) & (df['upper_BB'] < df['upper_KC'])
        df['squeeze_off'] = (df['lower_BB'] < df['lower_KC']) & (df['upper_BB'] > df['upper_KC'])
        return df

def madrid_ribbon(df):
  for i in range(5,105,5):
    df.ta.ema(i,append=True)

  for i in range(110,170,10):
    df.ta.ema(i,append=True)

  return df
    
def rsi(df):
    df.ta.stochrsi(append=True)
    df.ta.rsi(append=True)
    return df
    
def rsi_ma(df):
    return ta.sma(ta.rsi(df['close']),5)
    

In [71]:
class Stock_Detail:
    def __init__(self,symbol=None):
        self.symbol = symbol
        
    def Information(self,symbol):
        df = yf.download('{}.NS'.format(symbol), period='2d', interval = '1m',progress=False)
        df.columns = ['open','high','low','close','adj_close','volume']
        df = df[['open','high','low','close','volume']]
        df.reset_index(inplace=True)
        df = squeeze_momentum(df.copy())
        df = rsi(df.copy())
        df = madrid_ribbon(df.copy())
        df['rsi_ma'] = rsi_ma(df.copy())
        df['std'] = df.EMA_10.rolling(10).std()*100/df['close']
        df['avg_close_diff'] = ta.ema((1 - df['EMA_100'] / df['close']),2)
        df.dropna(axis=0,inplace=True)
        df = df.round(decimals=2)
        return df

In [72]:
symbol1 = list(pd.read_csv('/content/ind_nifty50list.csv')['Symbol'])
pd.DataFrame(symbol1,columns=['Company'])

,Company
0,ADANIPORTS
1,APOLLOHOSP
2,ASIANPAINT
3,AXISBANK
4,BAJAJ-AUTO
5,BAJFINANCE
6,BAJAJFINSV
7,BPCL
8,BHARTIARTL
9,BRITANNIA


In [73]:
def Final_Data():
  stock = Stock_Detail()
  empty = pd.DataFrame() 
  for i in range(0,len(symbol1)-3,3):
    last = pd.concat([stock.Information(symbol1[i]),stock.Information(symbol1[i+1]),empty], axis=0, ignore_index=True)
    empty = pd.concat([last,stock.Information(symbol1[i+2])], axis=0,ignore_index=True)
  return empty

In [74]:
Final_Data()

,Datetime,open,high,low,close,volume,upper_BB,lower_BB,tr0,tr1,...,EMA_100,EMA_110,EMA_120,EMA_130,EMA_140,EMA_150,EMA_160,rsi_ma,std,avg_close_diff
0,2022-08-01 11:54:00+05:30,1047.00,1047.60,1047.00,1047.00,11916,1047.64,1046.22,0.60,0.50,...,1048.02,1047.99,1047.99,1047.98,1047.99,1048.08,1048.16,42.61,0.00,-0.0
1,2022-08-01 11:55:00+05:30,1047.00,1048.25,1047.00,1048.25,11984,1047.82,1046.10,1.25,1.25,...,1048.02,1047.99,1047.99,1047.99,1047.99,1048.09,1048.17,46.59,0.01,-0.0
2,2022-08-01 11:56:00+05:30,1048.25,1050.10,1048.25,1050.00,8663,1048.66,1045.51,1.85,1.85,...,1048.06,1048.03,1048.02,1048.02,1048.02,1048.11,1048.19,53.14,0.02,0.0
3,2022-08-01 11:57:00+05:30,1050.00,1050.40,1049.80,1050.00,3974,1049.25,1045.22,0.60,0.40,...,1048.10,1048.07,1048.06,1048.05,1048.05,1048.14,1048.21,59.69,0.04,0.0
4,2022-08-01 11:58:00+05:30,1050.00,1050.00,1049.55,1049.65,4084,1049.64,1045.09,0.45,0.00,...,1048.13,1048.09,1048.08,1048.07,1048.07,1048.16,1048.23,64.72,0.05,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28370,2022-08-02 15:26:00+05:30,738.80,738.80,738.20,738.50,26461,741.19,738.69,0.60,0.00,...,740.74,740.75,740.76,740.77,740.79,740.81,740.84,38.57,0.03,-0.0
28371,2022-08-02 15:27:00+05:30,738.50,738.50,737.15,737.15,48287,741.41,738.09,1.35,0.00,...,740.67,740.68,740.70,740.72,740.74,740.76,740.79,35.58,0.05,-0.0
28372,2022-08-02 15:28:00+05:30,737.45,738.50,737.00,737.40,48578,741.33,737.77,1.50,1.35,...,740.60,740.63,740.65,740.66,740.69,740.71,740.75,33.69,0.06,-0.0
28373,2022-08-02 15:29:00+05:30,737.40,737.95,736.15,737.00,25658,741.43,737.33,1.80,0.55,...,740.53,740.56,740.58,740.61,740.64,740.67,740.70,30.30,0.08,-0.0


In [80]:
Final_Data().to_csv('data_sm.csv')